In [11]:
import requests
from requests import Response
import time
import pandas as pd

from pydantic import BaseModel, RootModel, Field

class BikeOfSite(BaseModel):
    行政區:str = Field(alias='sarea')
    站點名稱:str = Field(alias='sna')
    時間:str = Field(alias='mday')
    總車輛數:int = Field(alias='tot')
    可借:int = Field(alias='sbi')
    可還:int = Field(alias='bemp')

############################

class BikeOfSites(RootModel):
    root:list[BikeOfSite]

    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, idx):
        return self.root[idx]

############################

def get_response() -> Response | None:
    url_path:str = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

    count:int = 0
    limit:int = 3
    delay:int = 5

    while True and count < limit:
        try:
            print(f'執行第 {count + 1} 次下載:')
            response:Response = requests.get(url_path, timeout=15)
            response.raise_for_status()
        except Exception as e:
            count += 1
            print('連線失敗。')
            print(e)

            if count < limit:
                print(f'等待 {delay} 秒後再次嘗試。')
                time.sleep(delay)
        else:
            if response.status_code != 200:
                count += 1
                print('下載失敗。')
                print(response.reason)
                
                if count < limit:
                    print(f'等待 {delay} 秒後再次嘗試。')
                    time.sleep(delay)
            else:
                break
    
    if count >= limit:
        print(f'失敗已達 {limit} 次，不再執行下載動作。')
        return None
    else:
        print('下載成功')
        return response

############################

def jsonToList(raw:str) -> list[dict] | None:
    try:
        datas:BikeOfSites = BikeOfSites.model_validate_json(raw)
        items:list[dict] = datas.model_dump()
        return items
    except Exception as e:
        print('預想外的格式。')
        print(e)
        return None

############################
    
#若寫在function裡，例如main()，則pd.DataFrame()不會顯示Table
#沒有報錯，print()顯示有資料，原因不明
#20240402 使用 display() 可顯示資料, 僅限在 ipynb 裡
# def main():
#     response:Response | None = get_response()

#     if not response:
#         print('dsfghdfh111')
#         return
    
#     rawString:str = response.text
#     download_data:list[dict] | None = jsonToList(rawString)

#     if not download_data:
#         return

#     pd.DataFrame(download_data)
#     display(pd.DataFrame(download_data))
    
# main()

############################

response:Response | None = get_response()

# #寫在if判斷裡，pd.DataFrame()也不會顯示
# if response:
#     rawString:str = response.text
#     download_data:list[dict] | None = jsonToList(rawString)

#     if download_data:
#         pd.DataFrame(download_data)
#         print(len(download_data))

rawString:str = response.text

download_data:list[dict] | None = jsonToList(rawString)

pd.DataFrame(download_data)

執行第 1 次下載:
下載成功


,行政區,站點名稱,時間,總車輛數,可借,可還
0,大安區,YouBike2.0_捷運科技大樓站,2024-03-28 23:29:18,28,1,27
1,大安區,YouBike2.0_復興南路二段273號前,2024-03-28 23:15:14,21,1,20
2,大安區,YouBike2.0_國北教大實小東側門,2024-03-28 22:45:19,16,3,13
3,大安區,YouBike2.0_和平公園東側,2024-03-28 23:29:18,11,2,9
4,大安區,YouBike2.0_辛亥復興路口西北側,2024-03-28 23:28:14,16,1,15
...,...,...,...,...,...,...
1374,臺大公館校區,YouBike2.0_臺大總圖書館西南側,2024-03-28 22:13:17,30,1,29
1375,臺大公館校區,YouBike2.0_臺大黑森林西側,2024-03-28 23:28:14,20,1,19
1376,臺大公館校區,YouBike2.0_臺大獸醫館南側,2024-03-28 21:33:14,24,0,24
1377,臺大公館校區,YouBike2.0_臺大新體育館東南側,2024-03-28 22:21:14,40,0,40


In [3]:
def test123():
    testList = [
        {
            'aaa': 123,
            'bbb': 'aab'
        }
    ]

    pd.DataFrame(testList)

test123()

In [4]:
testList2 = [
    {
        'aaa': 123,
        'bbb': 'aab'
    }
]

pd.DataFrame(testList2)

,aaa,bbb
0,123,aab
